<a href="https://colab.research.google.com/github/gtbnhyujmj/-Good-Auto_Shorts_Maker/blob/main/%5BV2%5D_Stickered_Movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import random

# def Area

In [6]:
# ===== 函數：合成透明圖層（快速 numpy） =====
def blend_with_alpha_fast(base_img, overlay_imgs):
    base_rgba = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGBA)
    for overlay in overlay_imgs:
        overlay_rgba = np.array(overlay.convert('RGBA').resize((base_rgba.shape[1], base_rgba.shape[0]), Image.LANCZOS))
        alpha_overlay = overlay_rgba[..., 3:] / 255.0
        alpha_base = base_rgba[..., 3:] / 255.0
        out_alpha = alpha_overlay + alpha_base * (1 - alpha_overlay)
        base_rgba[..., :3] = (overlay_rgba[..., :3] * alpha_overlay + base_rgba[..., :3] * alpha_base * (1 - alpha_overlay)) / (out_alpha + 1e-6)
        base_rgba[..., 3:] = out_alpha * 255
    result_bgr = cv2.cvtColor(base_rgba, cv2.COLOR_RGBA2BGR)
    return result_bgr

In [3]:
# ===== 函數：處理影片，每幀合成 PNG 疊圖 =====
def overlay_multiple_pngs_on_video_fast(video_path, overlay_paths, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    overlay_images = [Image.open(path).resize((width, height), Image.LANCZOS) for path in overlay_paths]
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        result_frame = blend_with_alpha_fast(frame, overlay_images)
        out.write(result_frame)
    cap.release()
    out.release()
    print(f"✅ 影片已產出：{output_path}")

# Main Precessing

In [4]:
# ===== 資料夾設定 =====
BACKGROUND_DIR = "/content/drive/MyDrive/shorts/backgrounds_MP4"  # 可包含 MP4 或 PNG
CHARACTER_DIR = "/content/drive/MyDrive/shorts/characters"
OVERLAY_DIR = "/content/drive/MyDrive/shorts/PNG_Overlayers"
TEXT_STICKER_DIR = "/content/drive/MyDrive/shorts/Text_Overlayers"
OUTPUT_DIR = "/content/drive/MyDrive/shorts/movie01"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [7]:
# ===== 主程式流程：對每張貼紙產出影片 =====
text_files = [f for f in os.listdir(TEXT_STICKER_DIR) if f.endswith('.png')]
if not text_files:
    print("⚠️ 找不到任何貼紙，請確認 TEXT_STICKER_DIR 資料夾內是否有 PNG。")

for text_file in text_files:
    # 讀入文字貼紙
    text_path = os.path.join(TEXT_STICKER_DIR, text_file)

    # 隨機抽背景
    bg_list = [os.path.join(BACKGROUND_DIR, f) for f in os.listdir(BACKGROUND_DIR) if f.endswith(('.mp4', '.png'))]
    if not bg_list:
        print("❌ 沒有背景素材，請確認背景資料夾。")
        continue
    bg_path = random.choice(bg_list)

    # 隨機抽角色與特效
    def rand_img(folder):
        items = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png')]
        return random.choice(items) if items else None
    layers = [text_path]  # 最底層為文字貼紙
    for folder in [CHARACTER_DIR, OVERLAY_DIR]:
        picked = rand_img(folder)
        if picked:
            layers.append(picked)

    # 合成影片
    output_path = os.path.join(OUTPUT_DIR, text_file.replace(".png", ".mp4"))
    if bg_path.endswith('.mp4'):
        overlay_multiple_pngs_on_video_fast(bg_path, layers, output_path)
    else:
        # 處理靜態背景圖
        bg_img = cv2.imread(bg_path)
        bg_img = cv2.resize(bg_img, (1080, 1920))
        overlay_imgs = [Image.open(p).resize((1080, 1920), Image.LANCZOS) for p in layers]
        result = blend_with_alpha_fast(bg_img, overlay_imgs)
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (1080, 1920))
        for _ in range(75): out.write(result)
        out.release()
        print(f"✅ 靜態背景影片產出：{output_path}")

print("🎬 所有影片產出完成！")

✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_0.mp4
✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_1.mp4
✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_2.mp4
🎬 所有影片產出完成！
